In [42]:
!nvidia-smi

Tue Mar  1 15:10:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    37W / 250W |  15981MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [43]:
from google.colab import auth
auth.authenticate_user()

In [44]:
!gcloud config set project alecsharpie
!gcloud config list

Updated property [core/project].
[component_manager]
disable_update_check = True
[compute]
gce_metadata_read_timeout_sec = 0
[core]
account = alecsharpie@gmail.com
project = alecsharpie

Your active configuration is: [default]


In [45]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0   137k      0 --:--:-- --:--:-- --:--:--  137k
OK
69 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.40.0).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 69 not upgraded.


In [46]:
!mkdir gcs_bucket
!gcsfuse --implicit-dirs image-datasets-alecsharpie gcs_bucket

mkdir: cannot create directory ‘gcs_bucket’: File exists
2022/03/01 15:10:38.687281 Start gcsfuse/0.40.0 (Go version go1.17.6) for app "" using mount point: /content/gcs_bucket
2022/03/01 15:10:38.700758 Opening GCS connection...
2022/03/01 15:10:39.119961 Mounting file system "image-datasets-alecsharpie"...
2022/03/01 15:10:39.120898 File system has been successfully mounted.


In [47]:
!ls gcs_bucket

cassava_farmer	packages


In [48]:
#!pip install git+https://github.com/alecsharpie/cassava_farmer.git --no-cache


In [49]:
!mkdir history
!mkdir models

mkdir: cannot create directory ‘history’: File exists
mkdir: cannot create directory ‘models’: File exists


In [50]:
from tensorflow.keras.utils import image_dataset_from_directory

def get_image_generator_local(
    batch_size,
    train_path='raw_data/cassava-leaf-disease-classification/train_images_mid'
):


    train_ds = image_dataset_from_directory(
        train_path,
        batch_size=batch_size,
        subset='training',
        validation_split=.20,
        seed=42,
        image_size=(512, 512),
        shuffle = True
    )

    class_names = train_ds.class_names

    train_size = train_ds.cardinality().numpy()
    train_ds = train_ds.unbatch().batch(batch_size)
    train_ds = train_ds.repeat()

    val_ds = image_dataset_from_directory(train_path,
                                          batch_size=32,
                                          subset='validation',
                                          validation_split=.20,
                                          seed=42,
                                          image_size=(512, 512),
                                          shuffle = True)

    val_size = val_ds.cardinality().numpy()
    val_ds = val_ds.unbatch().batch(batch_size)
    val_ds = val_ds.repeat()
    return train_ds, train_size, val_ds, val_size


In [51]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras import layers
from tensorflow.keras import Sequential


#from tensorflow.config import run_functions_eagerly

#from google.cloud import storage


def build_aug_eff_model(input_shape, output_classes):

    augmentation = Sequential([
        layers.RandomContrast(0.2),
        layers.RandomRotation(40),
        layers.RandomTranslation(0, 0.2),
        layers.RandomTranslation(0.2, 0),
        layers.RandomZoom(0.2, 0.2),
        layers.RandomFlip(mode="horizontal")
    ])

    dummy_input = layers.Input(shape=input_shape)

    topless_efficient_net = EfficientNetB0(include_top=False,
                                        weights='imagenet',
                                        input_tensor=dummy_input,
                                        pooling='max')

    aug_eff_model = Sequential([
        layers.Resizing(512, 512),
        #augmentation,
        topless_efficient_net,
        layers.Dense(12, activation = 'relu'),
        layers.Dropout(0.2),
        layers.Dense(output_classes, activation='softmax')
    ])

    #top3_acc = functools.partial(top_k_categorical_accuracy, k=3)
    #top3_acc.__name__ = 'top3_acc'

    aug_eff_model.compile(optimizer='adam',
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])
                        #run_eagerly=True)

    aug_eff_model.build((None, 512, 512, 3))
    aug_eff_model.summary()

    aug_eff_model.layers[2].trainable = False

    return aug_eff_model

In [ ]:
#from cassava_farmer.data import get_image_generator_local
#from cassava_farmer.model import build_aug_eff_model, save_model_to_gcp
#from cassava_farmer.gcs import storage_upload_file, storage_upload_folder

from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

batch_size = 16

es = EarlyStopping(patience=20)

model = build_aug_eff_model((512, 512, 3), 5)

train_ds, train_size, val_ds, val_size = get_image_generator_local(batch_size, 'gcs_bucket/cassava_farmer/train_images')


steps_per_epoch = train_size // batch_size
validation_steps = val_size // batch_size

# steps_per_epoch = 10
# validation_steps = 3

print('steps_per_epoch: ', steps_per_epoch)
print('validation_steps: ', validation_steps)

count_map = {
    0: 1087,
    1: 2189,
    2: 2386,
    3: 13158,
    4: 2577
}

# balance dataset
avg_count = np.array(list(count_map.values())).mean()
class_weights = {k: (1 / v) * avg_count for k, v in count_map.items()}

history = model.fit(
  train_ds,
  epochs=50,
  #batch_size=batch_size,
  class_weight = class_weights,
  steps_per_epoch=steps_per_epoch,
  validation_data=val_ds,
  validation_steps=validation_steps,
  validation_batch_size=batch_size,
  callbacks=[es]).history


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing_4 (Resizing)       (None, 512, 512, 3)       0         
                                                                 
 efficientnetb0 (Functional)  (None, 1280)             4049571   
                                                                 
 dense_8 (Dense)             (None, 12)                15372     
                                                                 
 dropout_4 (Dropout)         (None, 12)                0         
                                                                 
 dense_9 (Dense)             (None, 5)                 65        
                                                                 
Total params: 4,065,008
Trainable params: 4,022,985
Non-trainable params: 42,023
_________________________________________________________________
Found 21407 files belonging to 5 classe

In [ ]:
#Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history['loss'], label = 'train')
plt.plot(history['val_loss'], label = 'val')
plt.legend();

In [ ]:
import json
from datetime import datetime

history_file_name = f'history/{datetime.now().strftime("history_%Y-%m-%d_%H-%M-%S")}.json'
out_file = open(history_file_name, "w")
json.dump(history, out_file, indent="")
out_file.close()
#storage_upload_file(history_file_name)
files.download(f"/content/{history_file_name}]")





#storage_upload_folder('models/aug_eff_model_test')

In [ ]:
print('max train accuracy', max(history['accuracy']))
print('max val accuracy', max(history['val_accuracy']))

In [ ]:
# from cassava_farmer.trainer import Trainer
# trainer = Trainer('colab')
# trainer.train(data_path = 'gcs_bucket/cassava_farmer/train_images')

In [ ]:
model.save('models/aug_eff_model')

!zip -r /content/models/eff_model_test_colab.zip /content/models/aug_eff_model

from google.colab import files
files.download("/content/models/eff_model_colab.zip")

In [ ]:
# # load previous history
# import json

# history = json.load(open('/content/history/history_2022-02-27_16-43-50'))

# history

In [ ]:
# home made model

from tensorflow.keras import layers
from tensorflow.keras import Sequential


def build_model():

  augmentation = Sequential([
          layers.RandomContrast(0.2),
          layers.RandomRotation(40),
          layers.RandomTranslation(0, 0.2),
          layers.RandomTranslation(0.2, 0),
          layers.RandomZoom(0.2, 0.2),
          layers.RandomFlip(mode="horizontal")
      ])


  model = Sequential([
                      layers.Resizing(512, 512),
                      augmentation,
                      layers.Conv2D(16, (5, 5), activation = 'relu'),
                      layers.MaxPooling2D(4),
                      layers.Conv2D(32, (5, 5), activation = 'relu'),
                      layers.MaxPooling2D(4),
                      layers.Conv2D(64, (5, 5), activation = 'relu'),
                      layers.MaxPooling2D(4),
                      layers.Flatten(),
                      layers.Dense(10, activation = 'relu'),
                      layers.Dropout(0.4),
                      layers.Dense(5, activation = 'softmax')
  ])

  model.compile(optimizer = 'adam',
                loss = 'sparse_categorical_crossentropy',
                metrics = 'accuracy')
  return model


In [ ]:
import numpy as np

label_map = {
    '0': 'cassava_bacterial_blight', #1087
    '1': 'cassava_brown_streak_disease', #2189
    '2': 'cassava_green_mottle', #2386
    '3': 'cassava_mosaic_disease', #13158
    '4': 'healthy' #2577
}

count_map = {
    '0': 1087,
    '1': 2189,
    '2': 2386,
    '3': 13158,
    '4': 2577
}

# balance dataset
avg_count = np.array(list(count_map.values())).mean()
class_weights = {k: (1 / v) * avg_count for k, v in count_map.items()}

In [ ]:
test_model = build_model()
test_model.build((None, 512, 512, 3))
test_model.summary()

In [ ]:
test_model = build_model()

# history = test_model.fit(
#                 train_ds,
#                 epochs=1,
#                 class_weights = class_weights
#                 #batch_size=batch_size,
#                 steps_per_epoch=steps_per_epoch,
#                 validation_data=val_ds,
#                 validation_steps=validation_steps,
#                 validation_batch_size=batch_size).history
#                 #callbacks=[es]